In [1]:
import numpy as np
import matplotlib.pyplot as plt

from Quaternion import Quaternion

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [64]:
def jeffery_omega(L, K, q, Omega, E):
    """
    L: (lambda^2-1)/(lambda^2+1)
    K: (kappa^2-1)/(kappa^2+1)
    q: quaternion representing current orientation
    Omega: vorticity (lab frame) 
    E: strain matrix (lab frame)

    Returns angular velocity of particle (body frame)
    """
    R = q.get_R()
    n1 = R[:,0]
    n2 = R[:,1]
    n3 = R[:,2]

    omega1 = n1.dot(Omega) + (L-K)/(L*K-1.) * (n2.dot(E.dot(n3)))
    omega2 = n2.dot(Omega) + L * (n1.dot(E.dot(n3)))
    omega3 = n3.dot(Omega) - K * (n1.dot(E.dot(n2)))

    return np.array([omega1, omega2, omega3])

def plot_projection(n3s):
    """
    n3s: (N, 3) array
    
    Plots sequence of x and y-components as line in plane.
    """
    ax=plt.subplot(1,1,1)
    ax.plot(n3s[:,0],n3s[:,1],lw=0.4)
    ax.plot(n3s[0,0],n3s[0,1],ls='none',marker='o')
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_aspect('equal')

In [ ]:
q = Quaternion(axis=[0,1,0], angle=np.pi/2)

dt = 0.001
N = 40000

l = 7
k = 1
L = (l**2-1)/(l**2+1)
K = (k**2-1)/(k**2+1)

Omega = np.array([0,0,-.5])
E = np.array([
    [0,.5,0],
    [.5,0,0],
    [0,0,0]
])

n3s = np.zeros( (N,3) )
for n in range(N):
    n3s[n,:] = q.get_R()[:,2]
    
    omega = jeffery_omega(L, K, q, Omega, E)
    qdot = 0.5 * omega.dot(q.get_G())
    q = q + dt*qdot
    q.normalize()

plot_projection(n3s)
plt.show()